# Exercises, practical from the Sante Fe Institute Renormalization MOOC
---
"a modern introduction to renormalization from a complex systems point of view"

<http://renorm.complexityexplorer.org/>

Taught by Simon DeDeo

## Contents
1. [Unit 1 &mdash; An Introduction to Coarse-Graining](#1)
2. [Unit 2 &mdash; Renormalizing Markov Chains](#2)
3. (Unit 3 has no exercises)
4. [Unit 4 &mdash; Renormalizing Cellular Automata](#3)
1. [Unit 5 &mdash; Practical for Renormalizing the Ising Model](#4)
2. [Unit 6 &mdash; Renormalizing the Creature (Krone-Rhodes)](#5)
3. [Unit 7 &mdash; Renormalizing the Thermal Plasma](#6)
4. [Unit 8 &mdash; Rate-Distortion Theory](#7)

<a id="1"></a>
# Unit 1: An Introduction to Coarse-Graining
---

In [1]:
data = [["a", 0.0777093], ["b", 0.01694125], ["c", 0.02509587], 
["d", 0.0415729],["e", 0.1293273], ["f", 0.02237207], ["g", 0.01869932], 
["h", 0.063514], ["i", 0.0705279], ["j", 0.00162383], ["k", 0.00598080], 
["l", 0.04025481], ["m", 0.0275251], ["n", 0.0702613], ["o", 0.0746462], 
["p", 0.01533419], ["q", 0.001168940], ["r", 0.0602125], ["s", 0.0617264], 
["t", 0.0869565], ["u", 0.02793712], ["v", 0.01067520], ["w", 0.0229406], 
["x", 0.001564180], ["y", 0.02368643], ["z", 0.001745021]]

#### Q1: what is the entropy of the distribution?

In [2]:
from math import log

distribution = dict(data) #dict for easier lookup below
letters = distribution.keys() 
values = distribution.values()

def entropy(input_list):
	return -1*sum([x*log(x,2) for x in input_list])

data_H = entropy(values)
print('Distribution entropy: ', data_H)

Distribution entropy:  4.181203619915858


#### Q2: Coarse-grain this distribution to “vowel” and “not-vowel” (i.e., map each of the letters into one of two categories). What is the entropy of this coarse-grained distribution? For simplicity, assume the vowels are just the letters “a”, “e”, “i”, “o” and “u”. Neglect letters that can sometimes function as a vowel, like “y”, or cases where a string of letters can indicate a vowel sound, as in “loose”.


In [3]:
vowels = ["a", "e", "i", "o", "u"]
notvowels = list(letters - vowels)
vowel_prob = [distribution[v] for v in vowels]
notvowel_prob = [distribution[nv] for nv in notvowels]

#by the coarse-graining axiom for entropy
cg_H = sum(vowel_prob)*entropy(vowel_prob) + sum(notvowel_prob)*(entropy(notvowel_prob))
# = 2.265361175872152
print('Vowel/Not-vowel distribution entropy :', cg_H)

Vowel/Not-vowel distribution entropy : 2.2653611758721515


#### Q3: What are some of the rules that might apply to this coarse-grained time-series? If you want to go further, “Part of Speech Taggers” are widespread online—see, for example, the tagger in the Python NLTK package, http://www.nltk.org/book/ch05.html.

<a id="2"></a>
# Unit 2: Renormalizing Markov Chains
---

#### **Q1**: Prove the existence of two-state slippy counters that do not have a corresponding two-state microtheory (i.e., that can’t be derived as a coarse-graining of a two-state Markov chain of any form).


For a two-state slippy counter (single-parameter MC) to have a corresponding two-state microtheory, there must exist a stochastic matrix $A$ that is the square root of slippy counter transition matrix, $T$.

$T$ is defined as:
$
\begin{bmatrix} 
\varepsilon & 1-\varepsilon \\
1-\varepsilon & \varepsilon 
\end{bmatrix}
$ $\ldots$ with $\varepsilon$ representing the "stay" probability.

We can set up $A$ as:
$ 
\begin{bmatrix} 
a & 1-b \\
1-a & b 
\end{bmatrix}
$

Squaring $A$ yields $A^2 =$
$
\begin{bmatrix} 
a^2+(1-b)(1-a) & a(1-b)+b(1-b) \\
a(1-a)+b(1-a) & (1-a)(1-b)+b^2 
\end{bmatrix}
$

Since $\varepsilon = a^2+(1-b)(1-a) = (1-a)(1-b)+b^2$, and $a > 0$, $b>0$, we know $a=b$

Setting the top-left position of both $A^2$ and $T$ to be equal, and subbing $b$ for $a$

$\varepsilon = a^2 + (1-a)^2$

$0 = 2a^2 - 2a + (1 - \varepsilon)$

$a = \frac{2 \pm \sqrt{4-4(2)(1-\varepsilon)}}{4}$

$a = 1/2 \pm 1/2\sqrt{1 - 2(1-\varepsilon)}$

$1 - 2(1-\varepsilon) > 0$ to avoid imaginary roots

$2\varepsilon > 1$

$\varepsilon > 1/2$

That is, if $\varepsilon < 1/2$ then $a$ (and $b$) are complex, meaning they can't correspond to a stochastic matrix. Any slippy counter with $\varepsilon < 1/2$ cannot have been generated by the coarse-graining of a slippy counter.

#### Q2: Experiment with the three-state case. Take a Markov Chain over three states, represent it as a matrix, and use a system like Mathematica or Python to find the fixed points by repeated multiplication of the matrix on itself. Confirm the result using theory: find the eigenvector corresponding to the eigenvalue equal to unity, and show that the columns of the fixed point Markov chain are equal to the components of the first eigenvector.

In [208]:
import numpy as np

#left (i.e. solumn) stochastic matrix for MC
three_state = np.array([0.1, 0.0, 0.9, 
                     0.5, 0.8, 0.05, 
                     0.4, 0.2, 0.05]).reshape(3,3)

long_run = np.linalg.matrix_power(three_state, 100)
vals, vecs = np.linalg.eig(three_state)
v = np.where( np.isclose(vals,1) )
stat = np.squeeze( vecs[:,v]/sum(vecs[:,v]) )
print("eigenvectors(columns):\n", vecs)
print("\neigenvalues:\n", vals)
print("\neigenvector corresponding to eigenvalue==1 (normalized to sum to 1):\n", stat)
print("\nlong run MC distribution:\n", long_run)
# print(long_run[:,0])

print('\nevec == long run distribution?: ', all( np.isclose(long_run[:,0], stat) ))

eigenvectors(columns):
 [[-0.80740505  0.32338083  0.58185935]
 [ 0.29847508  0.88929729 -0.78699396]
 [ 0.50892997  0.32338083  0.20513461]]

eigenvalues:
 [-0.46729515  1.          0.41729515]

eigenvector corresponding to eigenvalue==1 (normalized to sum to 1):
 [ 0.21052632  0.57894737  0.21052632]

long run MC distribution:
 [[ 0.21052632  0.21052632  0.21052632]
 [ 0.57894737  0.57894737  0.57894737]
 [ 0.21052632  0.21052632  0.21052632]]

evec == long run distribution?:  True


Note: I'm not sure why 'stat' is not a stochastic vector &mdash; I have to normalize here so it sums to 1

#### Q3: Consider two rock-paper-scissors playing robots. Each robot decides what to do next based on both its move, and the move its opponent made, the step before. 
#### A. Write down two decision rules for each robot. You can make them up yourself; for example, a robot might play “best response” to its opponent’s prior move; another robot might play noisy best-response (i.e., with some small probability to play something other than best response). Then, for example, if the system is in state {R,P}, and robot one plays best response, its next move will be S (since Scissors would have beaten its opponent). In general, this will be a 9x3 matrix: for each of the nine possible states at the previous step {RR, RP, RS, PR, PP, PS, SR, SP, SS}, the robot has some probability to emit R, P, or S. If you write the matrix as A ia , then A 1,2 , for example is the probability that if the previous state of play was RR, then robot A will play paper. You might find it easier to use a dictionary representation if you’re working in Python, rather than keeping track of matrices.


In [6]:
#Noisily play best response to opponent's previous move
r1 = 0.1 #rate player1 plays a random move
player1 = \
{'RR': (0.5*r1, 1-r1, 0.5*r1),
 'RP': (0.5*r1, 0.5*r1, 1-r1),
 'RS': (1-r1, 0.5*r1, 0.5*r1),
 'PR': (0.5*r1, 1-r1, 0.5*r1),
 'PP': (0.5*r1, 0.5*r1, 1-r1),
 'PS': (1-r1, 0.5*r1, 0.5*r1),
 'SR': (0.5*r1, 1-r1, 0.5*r1),
 'SP': (0.5*r1, 0.5*r1, 1-r1),
 'SS': (1-r1, 0.5*r1, 0.5*r1)}

#Noisily play win-stay, (lose/tie)-shift
r2 = 0.1 #rate player1 plays move outside strategy, i.e. shift on a win, stay on a loss
player2 = \
{'RR': (r2, 0.5*(1-r2), 0.5*(1-r2)),
 'RP': (1-r2, 0.5*r2, 0.5*r2),
 'RS': (r2, 0.5*(1-r2), 0.5*(1-r2)),
 'PR': (0.5*(1-r2), r2, 0.5*(1-r2)),
 'PP': (0.5*(1-r2), r2, 0.5*(1-r2)),
 'PS': (0.5*r2, 1-r2, 0.5*r2),
 'SR': (0.5*r2, 0.5*r2, 1-r2),
 'SP': (0.5*(1-r2), 0.5*(1-r2), r2),
 'SS': (0.5*(1-r2), 0.5*(1-r2), r2)}


#### B. Combine these two decision rules together to get the evolving Markov Chain for the pair. Technically is a bit tricky, because you have to convert the pair of output symbols (R, and R) to a single symbol in the larger list of 9 possible combinations. In general, the probability of going from state i to state j is equal to the product of A ia (“robot A sees i and plays move a”) and B ib (“robot B sees i and plays move b”), and where j is the combination of the a and b moves—equal to 3 × (a − 1) + b. For example, if j is equal to PS, then a = 2, and b = 3). Be careful to define the player B’s rules so that, for example PS means that he (player B) played S. Again, you might find it easier to use a dictionary representation if you work in Python.


In [213]:
probabilities = dict()
for key in player1.keys():
    probabilities[key] = np.concatenate(
        [player1[key][0]*np.array(player2[key]), 
         player1[key][1]*np.array(player2[key]), 
         player1[key][2]*np.array(player2[key])])

MC = np.array(list(probabilities.values())).transpose() 
#have to transpose because the dict values represent columns, but list() orders them as rows
print(MC)

[[ 0.005   0.045   0.09    0.0225  0.0225  0.045   0.0025  0.0225  0.405 ]
 [ 0.0225  0.0025  0.405   0.005   0.005   0.81    0.0025  0.0225  0.405 ]
 [ 0.0225  0.0025  0.405   0.0225  0.0225  0.045   0.045   0.005   0.09  ]
 [ 0.09    0.045   0.005   0.405   0.0225  0.0025  0.045   0.0225  0.0225]
 [ 0.405   0.0025  0.0225  0.09    0.005   0.045   0.045   0.0225  0.0225]
 [ 0.405   0.0025  0.0225  0.405   0.0225  0.0025  0.81    0.005   0.005 ]
 [ 0.005   0.81    0.005   0.0225  0.405   0.0025  0.0025  0.405   0.0225]
 [ 0.0225  0.045   0.0225  0.005   0.09    0.045   0.0025  0.405   0.0225]
 [ 0.0225  0.045   0.0225  0.0225  0.405   0.0025  0.045   0.09    0.005 ]]


#### C. Find the fixed points of this system. (In general, to make sure that you flow to the 2-dimensional subspace, you can add a small jitter probability epsilon to make sure that you don’t get stuck in “limit cycles”.) Which robot wins in the long run?

In [214]:
#By iterating the MC to find the fixed distribution...
MC_fixed = np.linalg.matrix_power(MC, 100)

#vector for who wins in each state
winner = np.array([0,-1,1,1,0,-1,-1,1,0]) # 1 = Player1 win, -1 = Player2 win, 0 = tie
longRunSum = sum(MC_fixed[:,0]*winner)
# print(MC_fixed[0])
# print(MC_fixed[0]*winner)
print(longRunSum, ': Player {}'.format(1 + int(longRunSum<0)), 'wins!')

-0.550607287449 : Player 2 wins!


In [219]:
#Verify by finding the eigenvector of eigenvalue 1 and normalizing
vals, vecs = np.linalg.eig(MC)

v = np.where( np.isclose(vals,1) )
stat = np.squeeze( vecs[:,v]/sum(vecs[:,v]) )
# evals = np.linalg.eigvals(MC)
# evals
# np.where(evals[0] == 1)
winner = np.array([0,-1,1,1,0,-1,-1,1,0]) # 1 = Player1 win, -1 = Player2 win, 0 = tie
longRunSum = sum(stat*winner)
print(longRunSum, ': Player {}'.format(1 + int(longRunSum<0)), 'wins!')

(-0.550607287449+0j) : Player 2 wins!


<a id="3"></a>
# Unit 4: Renormalizing Cellular Automata
---

#### Q1: demonstrate the renormalization of Rule 105 to Rule 150. What other projections work? You’ll almost certainly want to code this up, but there’s no reason you can’t do it by hand with lots of paper. If you do try to code, here’s a guide for Python.

(in the following, De Deo's guide is commented with `##`; my code is interspersed where relevant.)

In [3]:
## First decide on a representation of the fine-grained rule-set (the lookup table f). In Python
## language, a simple choice would be an eight element list, where the first element is the output
## for the {0,0,0} state, the second for the {0,0,1} state, and so forth. Note that Wolfram’s
## definition of rules is a bit funny; Rule 0, for example, takes 000 (all black states) to 1 (white).
## The easiest way to get this is by converting the number to binary, and then reversing the
## bits—so, e.g., for Rule 105, the binary expansion is 01101001. If you reverse this, you get 10010110. 
## An input cell set “black, white, black” is then 010, or two; the third bit in the
## string (or, second counting from zero) is 0, so “black, white, black” goes to “black”.
## To access the lookup table, you’ll generically need to convert a list of 1s and 0s to the corresponding
## integer. In Python, you can use the ”int” function; for example, the list position
## corresponding to the state {0,1,1} is 3: int(’010’, 2) gives 2. So in Python, you
## can figure out the evolution of Rule 105 given “black, white, black” initial conditions as
## ’10010110’[int(’010’, 2)].

def rule(rulenum):
    b = format(rulenum, '08b') #convert to binary
    return b[::-1] #reverse the binary rep

## Then build a function that maps a lookup table f to the two-step evolution on the fine-grained
## scale. If the states at the micro-level are denoted {a, b, c, d, e, f}, then the left and right
## elements of the super-cell at time t+1 are given by A = f(f(a, b, c), f(b, c, d), f(c, d, e)) and
## B = f(f(b, c, d), f(c, d, e), f(d, e, f)).

def evolve(table, states, timesteps=1):
    currState = ''.join([table[int(states[i:i+3],2)] for i in range(len(states)-2)])
    if timesteps == 1:
        return currState
    else:
        return evolve(table, currState, timesteps-1)

## Decide on a representation for the projection operator. Again, in python language, a simple
## choice is a four element list, where the first element is the projection of {0,0}, the second of
## {0,1}, and so forth.

def project(cells, operator):
    #number of supercells
    sc = range(int(len(cells)/2))
    return ''.join([operator[int(cells[x*2:(x+1)*2], 2)] for x in sc])

## With these in place, you can take an f (the fine grained rule), a p (the projection lookup table),
## and a g (another rule, with the same 8-element lookup table), and determine whether the diagram
## commutes. We need only determine whether p(A, B) is equal to g(P(a, b), P(c, d), P(e, f))
## for all 2^6 values of {a, b, c, d, e, f}.

## To cycle through the different values, it’s simplest to take a for-loop and convert the decimal
## values back to a string representation. In python, you can use the string format (definitely
## not optimized for speed): [[j for j in ’{0:b}’.format(i).rjust(6, ’0’)] for i in
## range(2**6)]

## Using the f and g corresponding to rule 105 and 150, along with the ”edge detector p”, you
## should be able to confirm the result of the lecture.

startStates = [format(j,'06b') for j in range(2**6)]
p = '1001'
r105 = rule(105)
r150 = rule(150)

# evolved = [evolve(r105, S, 2) for S in startStates]
project_after_evolve = [project(evolve(r105, s, 2), p) for s in startStates]
# projected = [project(S, P) for S in startStates]
evolve_after_project = [evolve(r150, project(s, p)) for s in startStates]

#check if they match (commute)
print('f:rule 105, g:rule 150 : commutes? ', project_after_evolve == evolve_after_project)

f:rule 105, g:rule 150 : commutes?  True


In [47]:
## The final step in their analysis is “simple”: for each f, cycle through the 2^4 possibilities
## for p, and the 2^8 possibilities for g, to find choices of g and p that enable the diagram to
## commute. For p, don’t forget that you’ll want to avoid the ”trivial” projections—[0,0,0,0] and
## [1,1,1,1]—which are easy to avoid by setting the range from 1 to 2^4 − 1.

P = [format(x, '04b') for x in range(1, (2**4)-1)] 
F = G = [rule(x) for x in range(0, 256)]
S = [format(j,'06b') for j in range(2**6)]

commutes = dict.fromkeys(range(0,256))

def commute(f, g, p, s):
    return project(evolve(f, s, 2), p) == evolve(g, project(s, p))

def checkProjection(f, g, p):
    for s in S:
        if not commute(f, g, p, s):
            return False
    return True


def checkRule(f,g):
    projs = [p for p in P if checkProjection(f,g,p)]
    if projs:
        return projs
    return False

for f in F:
#     matches = [{int(g,2): checkRule(f, g)} for g in G if checkRule(f, g)]
    matches = {int(g,2): checkRule(f, g) for g in G if checkRule(f, g)}
    if matches:
        commutes[int(f,2)] = matches
    
# print(commutes)

{0: {0: ['0001', '0010', '0011', '0100', '0101', '0110', '0111'], 255: ['1000', '1001', '1010', '1011', '1100', '1101', '1110']}, 1: {1: ['0001'], 127: ['1110']}, 2: {0: ['0001', '0010', '0011', '0100', '0101', '0110', '0111'], 255: ['1000', '1001', '1010', '1011', '1100', '1101', '1110']}, 3: {3: ['0001'], 63: ['1110']}, 4: {0: ['0001'], 255: ['1110']}, 5: {1: ['0001', '0011', '0101'], 127: ['1010', '1100', '1110']}, 6: None, 7: None, 8: {0: ['0001'], 255: ['1110']}, 9: {1: ['0001'], 127: ['1110']}, 10: {12: ['0011', '0101'], 207: ['1010', '1100']}, 11: {12: ['0100'], 207: ['1011']}, 12: {0: ['0001'], 15: ['0010', '1101'], 255: ['1110']}, 13: {1: ['0001'], 127: ['1110']}, 14: {15: ['0010', '1101']}, 15: {15: ['0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110']}, 16: {0: ['0001', '0010', '0011', '0100', '0101', '0110', '0111'], 255: ['1000', '1001', '1010', '1011', '1100', '1101', '1110']}, 17: {17: ['0001'], 119: ['1110']}, 1

In [2]:
# let's take a look at the f, g, p relationships
# a hack way to legibly print dictionaries
import json
print('READ BELOW AS: \n \
{F : \n  \
    \t{G : [ \n \
        \t\tP1, \n \
        \t\tP2, \n \
        \t\t...,\n \
        \t\tPn \n \
        \t\t]\n \
    \t}\n \
}\n')
print('------------------------------ \n')
print(json.dumps(commutes, indent=4))


READ BELOW AS: 
 {F : 
      	{G : [ 
         		P1, 
         		P2, 
         		...,
         		Pn 
         		]
     	}
 }

------------------------------ 



NameError: name 'commutes' is not defined

<a id="5"></a>
# Unit 5: Ising practical 
---

#### Suggest a social or biological system where the internal structure of a component might be a two dimensional vector with unrestricted length.

#### What network might describe the Ising model at a cocktail party, where guests are perfectly mingling?

#### If you are an evolutionary biologist, what network structures might be more or less amenable to the emergence of cooperation and group selection?

#### In words, what configurations (choices for the values of the nodes) give you lower energy? What give you higher energy?

Lower energy states have more interacting cells in the same spin state. Higher energy states have more interacting cells in unlike spin states.

#### Take $N$ to be three, and have all the nodes connected to each other. Write down the matrix $J_{ij}$ . What is the energy of such a system when $σ_1$, $σ_2$ and $σ_3$ are all $+1$? What about when they are all $−1$? 

In [62]:
import numpy as np
import itertools

shape = (3,1)
sigma = np.ones( shape, dtype=int)
J = np.triu(np.ones( (sigma.size, sigma.size) , dtype=int)) - np.identity(sigma.size, dtype=int) #row to column

def energy(state, J):
    return -1*sum([J[i][j]*state[i]*state[j] for i in range(state.size) for j in range(state.size)])

print('J: \n', J, '\n')
for s in itertools.product([-1,1],repeat=3):
    print('energy of state',s,'= ', energy(np.array(s),J)) 

J: 
 [[0 1 1]
 [0 0 1]
 [0 0 0]] 

energy of state (-1, -1, -1) =  -3
energy of state (-1, -1, 1) =  1
energy of state (-1, 1, -1) =  1
energy of state (-1, 1, 1) =  1
energy of state (1, -1, -1) =  1
energy of state (1, -1, 1) =  1
energy of state (1, 1, -1) =  1
energy of state (1, 1, 1) =  -3


#### What is the symmetry of this interaction? How could you alter $H$ to break it?

For $n \leq N$, states with $n$ cells spin-up have equal energy, and they have the same energy as all states with $n$ cells spin-down.

In [65]:
#An energy function that would break this symmetry:
def energyBreak(state, J):
    #must "treat" +1,-1 differently
    return -1*sum([J[i][j]*(state[i]+state[j]) for i in range(state.size) for j in range(state.size)])

for s in itertools.product([-1,1],repeat=3):
    print('energy\' of state',s,'= ', energyBreak(np.array(s),J)) 


energy' of state (-1, -1, -1) =  6
energy' of state (-1, -1, 1) =  2
energy' of state (-1, 1, -1) =  2
energy' of state (-1, 1, 1) =  -2
energy' of state (1, -1, -1) =  2
energy' of state (1, -1, 1) =  -2
energy' of state (1, 1, -1) =  -2
energy' of state (1, 1, 1) =  -6


This function still has symmetry across states with the same number and sign of cells, but breaks the symmetry across states with equal number of same-sign cells

e.g. $H(-1,1,1)$ no longer equals $H(1,-1,-1)$, but still equals $H(1,-1,1)$

#### A state space with +1 and −1 can at best have an $H$ symmetric on switching sign. Pick a state space that is more interesting than {+1, −1}. How about one that can have more interesting symmetries? How about a system with $n$ states – now it can be symmetric under the $n!$ permutations? 

#### Take n to be three, and write an H that breaks permutation symmetry on {red, black, yellow}. Can you find an H that breaks permutation symmetry to a smaller symmetry group – say, to a permutation on only two elements? Can you think of a friend who has this reduced symmetry? 

#### Read up on the groups $S_n$ (all permutations on $n$ elements) and $A_n$ (the alternating group on $n$ elements.) Take $n$ to be four, and write an $H$ that breaks $S_4$ to $A_4$ . What happens if you try to do this in the $n = 3$ case? ($S_3$ to $A_3$) 

#### How about a continuous state space, perhaps symmetric under O(2), or SU (3)?

#### Given the $N = 3$ system above, what is $P(\{+1, +1, +1\})$ when $β$ is 1, or 10, or 0.1? How does it compare to $P(\{+1, +1, −1\})$? What does this tell you about the relation between order and (inverse) temperature? 

#### Recall the definition of entropy. How does the entropy, $S$, of the $N = 3$ system change as a function of $β$? 

In [76]:
import itertools
def boltzmann(state, J, beta):
    Z = sum([np.exp(-beta*energy(np.array(sig), J)) for sig in itertools.product([-1,1],repeat=state.size)])
    return np.exp(-beta*energy(state, J))/Z
    
sigma = np.array([1, 1, 1])
# print('+1,+1,+1:',[boltzmann(sigma, J, 0.1), boltzmann(sigma, J, 1), boltzmann(sigma, J, 10)])
sigma1 = np.array([1, 1, -1])
# print('+1,+1,-1:',[boltzmann(sigma1, J, 0.1), boltzmann(sigma1, J, 1), boltzmann(sigma1, J, 10)])

for s in itertools.product([-1,1],repeat=3):
    print(s, [boltzmann(np.array(s), J, -10), boltzmann(np.array(s), J, -1), boltzmann(np.array(s), J, -0.1), boltzmann(np.array(s), J, -0.01)])


distribution = {s:boltzmann(np.array(s), J, 0.9) for s in itertools.product([-1,1],repeat=3)}

# entropy(distribution.values())
# print(entropy()
# print([boltzmann(np.array(s), J, 0.9) for s in itertools.product([-1,1],repeat=3)])
for beta in range(-10,11):
    print('beta:',round(0.1*beta, 2), 'entropy: ', entropy([boltzmann(np.array(s), J, 0.1*beta) for s in itertools.product([-1,1],repeat=3)]))

(-1, -1, -1) [7.0805904254859815e-19, 0.0030340827600584424, 0.091316293623993447, 0.12128761878313185]
(-1, -1, 1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(-1, 1, -1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(-1, 1, 1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(1, -1, -1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(1, -1, 1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(1, 1, -1) [0.16666666666666666, 0.16565530574664719, 0.1362279021253355, 0.1262374604056227]
(1, 1, 1) [7.0805904254859815e-19, 0.0030340827600584424, 0.091316293623993447, 0.12128761878313185]
beta: -1.0 entropy:  2.6287617321
beta: -0.9 entropy:  2.64491976491
beta: -0.8 entropy:  2.66632003881
beta: -0.7 entropy:  2.69416843702
beta: -0.6 entropy:  2.72957147286
beta: -0.5 entropy:  2.77316624405
beta: -0.4 entropy:

Beta decreases from 3 to 1 as beta increases from 0 to inf. Intuitively this makes sense, as non-interacting cells ($\beta = 0$) are equivalent to $3$ independent random variables with equal probability across two values, or 3 bits of information, whereas high beta values ($\beta > 2.1$) push the nearly all the probability mass evenly into the $+1,+1,+1$ and $-1,-1,-1$ states, which is equivalent to a single random variable with two equally probable outcomes, or one bit of information.

As beta decreases from 0, the entropy values approach $\approx 2.585$. In this case, increasing negative beta pushes the probability mass from the most ordered to (evenly over) the most disordered states

$-\sum_\sigma (P(\sigma)\cdot \log_{2}(P(\sigma))) \approx 2.585$ where $P(\{-1,-1,-1\}) \approx 0$ and $P(\{+1,+1,+1\}) \approx 0$ and for any other state $\approx 1/6$

#### How about the heat capacity, defined as $dS/dβ$? Where is the maximum of the heat capacity? Can you see how to use $Z(β)$ to compute quantities like entropy?

#### Let’s start with the Ising model. Consider the contribution to the energy from a single (arbitrarily chosen) spin $σ_a$ , $H_{a} = -\sigma_{a} \sum_{i} J_{ia} \sigma_{i}$ . Derive this from Eq. 1, or, conversely, convince yourself that $H$ is a $\sum_{a} H_{a}$ 


$H(\{\sigma_{i}\}) = - \sum_{i,j}J_{ij}\sigma_{i}\sigma_{j}$

$H(\sigma_{a}) = - \sum_{i,a}J_{ia}\sigma_{i}\sigma_{a}$

$H(\sigma_{a}) = - \sigma_{a} \sum_{i}J_{ia}\sigma_{i}$

#### What is $n_c$ on a two-dimensional lattice (such as the lines on a Go board, or the street corners in uptown Manhattan.) What is $n_c$ on a fully connected graph? What is $n_{c}$ on the small world network? 

$ H_a = −n_{c}σ_{a}σ̄$

- On a two-dimensional lattice $n_{c}=4$, assuming the lattice is infinite or has a left-right, top-bottom "wrap around" topology. 

- For a Go board or similar $i \times j$ lattice with boundaries, the $(i-2)\times(j-2)$ center cells have $n_{c}=4$, the $2(j-2) + 2(i-2)$ side cells have $n_{c}=3$, and the 4 corner cells have $n_{c}=2$. 

- On a fully connected graph, $n_{c}=N-1$.

- On a small-world network, $n_{c}$ is a value from the degree distribution.

<a id="6"></a>
# Unit 6: Renormalizing the Creature (Krohn-Rhodes Theorem) 
---

#### Consider now the “cyclic” group $Z_6$ . In our language, $Z_6$ is a creature with six internal states, and a single operator, C. If we number the states, then C takes 1 → 2, 2 → 3, 3 → 4, 4 → 5, 5 → 6, and 6 → 1. (So you can think of it as a counter that only goes up to six before jumping back to one.)

#### Q1: Find a coarse-graining of $Z_6$ with two states. Hint: you might find it helpful to draw the transitions in a $2 × 3$ grid. 

$\text{Odd} = \{1,3,5\}$ 

$\text{Even} = \{2,4,6\}$

$C : O \rightarrow E$

$C : E \rightarrow O$

(notation probably butchered!)

#### Q2: Find a coarse-graining of $Z_{36}$ with 3 states.

$R_1 = \{1,4,7, \ldots , 34\}$

$R_2 = \{2,5,8, \ldots , 35\}$

$R_0 = \{3,6,9, \ldots , 36\}$ 

$C : R_1 \rightarrow R_2$

$C : R_2 \rightarrow R_0$

$C : R_0 \rightarrow R_1$

#### Q3: Explain why $Z_{13}$ doesn’t have a (deterministic) coarse-graining.

There is no way to divide a prime number of states into subsets of equal size &mdash; the size and number of subsets would be the number's factors, if so &mdash; which is required of a coarse-graining that has a single deterministic transition.

<a id="7"></a>
# Unit 7: Renormalizing the Thermal Plasma 

### Q1: Plot the effective electric charge at some fixed distance r, as a function of temperature, and plasma density. How do these effects compete?

### Q2: If you have a background in physics, work through the full derivation of the Debye length using Richard Fitzpatrick’s material (see recommended reading). My blackboard work here is a partial account of that derivation.

<a id="8"></a>
# Unit 8: Rate-Distortion Theory: Keeping the Things that Matter 

### Q1: Write down a distortion matrix that leads you to prefer the two-state model. Write down a distortion matrix that leads you to prefer the original three-state coarse-graining. In this case, don’t worry about the mutual information term; just find a distortion matrix that makes one preferable to the other as a representation of the system.